In [ ]:
path = './data/qoe/federated_devices/pokemon_complet.csv'
df = read_csv(path)

train_set=df.sample(frac=0.8,random_state=200) #random state is a seed value
test_set=df.drop(train_set.index)
# test data
test_dl = DataLoader(test_set, batch_size=32, shuffle=True)

# slipt the data into 3 devices dataset
train_set1 =  train_set.loc[train_set['QoD_model'].isin([5, 8, 9])]
train_set2 =  train_set.loc[train_set['QoD_model'].isin([2, 1, 9])]
train_set3 =  train_set.loc[train_set['QoD_model'].isin([3, 4, 6])]

# device1 dataset
train_set1.rename(columns=train_set1.iloc[0]).drop(train_set1.index[0])
train_set_data1  = train_set1.iloc[:, 0:16].astype('float32')
target_set_data1 = train_set1.iloc[:, 16:21].astype('float32')
train_tensor1 = torch.from_numpy(train_set_data1.to_numpy()).float()
target_tensor1  = torch.from_numpy(target_set_data1.to_numpy()).float()

# device2 dataset
train_set2.rename(columns=train_set2.iloc[0]).drop(train_set2.index[0])
train_set_data2  = train_set2.iloc[:, 0:16].astype('float32')
target_set_data2 = train_set2.iloc[:, 16:21].astype('float32')
train_tensor2 = torch.from_numpy(train_set_data2.to_numpy()).float()
target_tensor2  = torch.from_numpy(target_set_data2.to_numpy()).float()

# device3 dataset
train_set3.rename(columns=train_set3.iloc[0]).drop(train_set3.index[0])
train_set_data3  = train_set3.iloc[:, 0:16].astype('float32')
target_set_data3 = train_set3.iloc[:, 16:21].astype('float32')
train_tensor3 = torch.from_numpy(train_set_data3.to_numpy()).float()
target_tensor3  = torch.from_numpy(target_set_data3.to_numpy()).float()

# Create the federated enviroment 
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
joe = sy.VirtualWorker(hook, id="joe")


# creating the federated dataset 
target_tensor_1 = target_tensor1.type(torch.LongTensor)
target_tensor_2 = target_tensor2.type(torch.LongTensor)
target_tensor_3 = target_tensor3.type(torch.LongTensor)

# sending the data set to the devices
bob_train_dataset = sy.BaseDataset(train_tensor1,target_tensor1).send(bob) 
anne_train_dataset = sy.BaseDataset(train_tensor2, target_tensor2).send(alice)
joe_train_dataset = sy.BaseDataset(train_tensor3, target_tensor3).send(joe)

federated_train_dataset = sy.FederatedDataset([bob_train_dataset, anne_train_dataset,joe_train_dataset]) 
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, shuffle=False, batch_size=32)

model_master = MLP(16).to(device)

def train(model, federate_train_loader, optimizer, epoch):
    model.train()
    #criterion = MSELoss()
    criterion = L1Loss()
    for batch_idx, (data, target) in enumerate(federate_train_loader):
        model.send(data.location)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target.float())
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step()
        model.get()
        if batch_idx % 10 == 0:
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, 
                    batch_idx * 32 , # no of images done
                    len(federate_train_loader) * 32, # total images left
                    100. * batch_idx / len(federate_train_loader), 
                    loss.item()
                 )
            )


#Train the model 
optimizer = optim.SGD(model3.parameters(), lr=0.03)  
epoch = 100
for epoch in range(1, epoch + 1):
    train(model3, federated_train_loader, optimizer, epoch)
    
    
# save the model
import torch 
torch.save(model3.state_dict(), './model/master_model.pt')

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# pytorch mlp for regression
from numpy import vstack
from numpy import sqrt
from sklearn.metrics import mean_squared_error

# evaluate the model
def test(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        output = model(inputs)
        # retrieve numpy array
        output = output.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 5))
        # store
        predictions.append(output)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse
